### 필요한 라이브러리 호출


In [1]:
#오디오 처리 라이브러리
import librosa
import librosa.display as dsp
from IPython.display import Audio

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
#데이터 전처리

import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [5]:
#모델 재현성을 위하여 random seed고정
import random

def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(929)

### 데이터 불러오기

In [6]:
train = pd.read_csv('/content/gdrive/MyDrive/drive-download-20220614T024615Z-001/train.csv')
train.head()

,file_name,label
0,001.wav,9
1,002.wav,0
2,004.wav,1
3,005.wav,8
4,006.wav,0


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  400 non-null    object
 1   label      400 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 6.4+ KB


### 데이터 전처리

- 소리는 보통 특정 주파수를 갖느 sin함수

- 주파수 분석 기법 활용

- 아날로그 데이터를 디지털 신호로 변환

In [8]:
data,sample_rate = librosa.load('/content/gdrive/MyDrive/Dacon/train/001.wav',sr=16000)
print('sample_rate:', sample_rate, ',audio shape:', data.shape)
print('length:', data.shape[0]/float(sample_rate), 'secs')

sample_rate: 16000 ,audio shape: (10192,)
length: 0.637 secs


sampling rate의 의미는 초당 16000개(16000Hz 주파수)의 샘플을 가지고 있는 데이터라는 의미입니다. (1초에 음성 신호를 16000번 sampling)

sampling rate의 defult값은 22050Hz인데, 16000Hz으로 설정한 이유는 사람의 목소리는 대부분 16000Hz 안에 포함된다고 합니다.

또한 audio shape와 sampling rate를 이용해서 오디오 길이 계산을 할 수 있습니다.

In [13]:
#train_wav데이터 프레임 생성
def train_dataset():
    folder = "/content/gdrive/MyDrive/Dacon/train"
    dataset = []
    for file in tqdm(os.listdir(folder),colour='green'):
        if 'wav' in file:
            abs_file_path = os.path.join(folder,file)
            data, sr = librosa.load(abs_file_path, sr = 16000)
            class_label = int(train[train.file_name == file].label)
            dataset.append([data,class_label])
    
    print("Dataset 생성 완료")
    return pd.DataFrame(dataset,columns=['data','label'])

In [14]:
train_wav = train_dataset()

100%|██████████| 400/400 [00:19<00:00, 20.57it/s]

Dataset 생성 완료


In [15]:
train_wav.head()

,data,label
0,"[3.6655838e-05, -3.7366447e-06, 3.4776433e-05,...",5
1,"[0.00011985076, 0.00016174652, 0.00017246709, ...",9
2,"[-0.00043892296, -0.00073743664, -0.0006132907...",8
3,"[-4.289015e-05, 9.891299e-05, 2.6636611e-05, 0...",4
4,"[1.2653453e-05, 2.3892262e-05, -7.51332e-06, 4...",0


In [17]:
train_wav.to_csv("train_wav.csv")

### 음성 데이터 특징 추출

- 퓨리에 변환

    - 입력 신호를 다양한 주파수를 가지는 주기함수들로 분해

    - 노이즈 및 배경 소리로부터 실제로 유용한 소리의 특징 추출

- MFCC

   - 음성데이터를 특징벡터화

   - 사람이 인지하기 좋은 20~40ms로 나눠준다

- Mel-scale

   - 주파수 간격이 넓어지면 감지 못함
   - filter, scaling

- Argument

   - y : audio data
 
   - sr : sampling rate

   - n_mfcc : return 될 mfcc의 개수를 정해주는 파라미터. 더 다양한 데이터 특징을 추출하기 위해서 증가 시킵니다.

   - n_fft : frame의 length를 결정하는 파라미터. n_fft를 설정하면 window size가 디폴트 값으로 n_fft가 됩니다.

   - 사람의 목소리는 대부분 16000Hz 안에 포함이 되는데, 일반적으로 자연어 처리에서는 음성을 25m의 크기를 기본으로 하고 있습니다.

(ex. 16000Hz인 음성에서는 25m의 음성의 크기를 가지고 있으면 n_fft는 16000 * 0.025 = 400 (sampling rate * frame_length = n_fft)가 됩니다.)


   - hop_length : 윈도우 길이를 나타냅니다. 길이만큼 옆으로 가면서 데이터를 읽습니다.

   - hop_length도 마찬가지로 window 간의 거리이므로 sampling rate * frame_stride 가 됩니다.

In [21]:
def extract_features(file):
    audio, sample_rate = librosa.load(file, sr = 16000)
    extracted_features = librosa.feature.mfcc(y=audio,
                                              sr=sample_rate,
                                              n_mfcc=40)

    extracted_features = np.mean(extracted_features.T,axis=0)
    return extracted_features


In [22]:
extract_features('/content/gdrive/MyDrive/Dacon/train/001.wav')

array([-5.4157184e+02,  1.0199717e+02, -1.0018574e+01,  4.5054619e+01,
        7.3112831e+00,  1.0971639e+01, -1.2032939e+01, -5.8687963e+00,
       -1.8881397e+00,  4.2930884e+00, -8.1847525e+00, -2.3072267e+00,
       -9.1721897e+00,  1.4182716e+01, -1.2839543e+01, -3.1000307e+00,
       -3.0502689e+00, -2.1911802e+00, -6.2639456e+00, -5.1691580e+00,
       -1.3974123e+01,  3.3810470e+00, -6.9977813e+00,  3.7736315e-01,
       -4.4287405e+00,  1.0799457e+00, -1.3639281e+00,  4.2418456e+00,
        2.3687005e+00,  2.8972096e+00,  2.6670651e+00,  1.8590584e+00,
       -3.8219376e+00, -1.6171500e-01, -1.4186366e+00, -4.1422081e+00,
       -5.3374414e+00, -9.9907333e-01, -3.3392251e+00, -4.2987290e-01],
      dtype=float32)

In [25]:
def preprocess_train_dataset(data):
    mfccs = []
    for i in data:
        extracted_features = librosa.feature.mfcc(y=i,
                                              sr=16000,
                                              n_mfcc=40)
        extracted_features = np.mean(extracted_features.T,axis=0)
        mfccs.append(extracted_features)
            
    return mfccs

mfccs = preprocess_train_dataset(train_wav.data)
mfccs = np.array(mfccs)

In [26]:
mfccs

array([[-5.0092383e+02,  1.0522231e+02, -3.0754614e+01, ...,
         2.7968860e+00,  2.0259073e+00,  9.0582495e+00],
       [-5.4157184e+02,  1.0199717e+02, -1.0018574e+01, ...,
        -9.9907333e-01, -3.3392251e+00, -4.2987290e-01],
       [-5.4121655e+02,  6.4172516e+01,  1.1352416e+01, ...,
         9.8170495e-01, -1.2399018e+00,  1.7476790e+00],
       ...,
       [-6.4978589e+02,  8.1067482e+01,  2.5364252e+01, ...,
         3.1881747e+00, -3.2863743e+00,  9.8691732e-01],
       [-5.7801672e+02,  8.9649864e+01,  6.7791176e+00, ...,
         3.5183573e+00, -6.8778259e-01,  2.9434044e+00],
       [-5.1473218e+02,  6.9641624e+01, -2.2522993e+01, ...,
        -8.1666964e-01, -7.3845682e+00, -2.4838426e+00]], dtype=float32)

### 변수 및 모델 정의

In [28]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(mfccs, train_wav.label, test_size=0.4)

In [29]:
print('학습시킬 train셋:', train_X.shape, train_y.shape)
print('검증할 val 셋:', test_X.shape, test_y.shape)

학습시킬 train셋: (240, 40) (240,)
검증할 val 셋: (160, 40) (160,)


In [30]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

model.fit(train_X, train_y)

RandomForestClassifier()

### 모델 성눙

In [31]:
import numpy as np

def ACCURACY(true, pred):
    score= np.mean(true==pred)
    return score

In [32]:
prediction = model.predict(test_X)

score = ACCURACY(test_y,  prediction)

print(f'모델의 정확도는 {score*100:.2f}% 입니다')

모델의 정확도는 63.12% 입니다


### test.csv분류하기

In [33]:
test = pd.read_csv("/content/gdrive/MyDrive/drive-download-20220614T024615Z-001/test.csv")
test.head()

,file_name
0,003.wav
1,008.wav
2,010.wav
3,015.wav
4,024.wav


In [34]:
def test_dataset():
    folder = '/content/gdrive/MyDrive/test'
    dataset=[]
    for file in tqdm(os.listdir(folder),colour='green'):
        if 'wav' in file:
            abs_file_path = os.path.join(folder,file)
            data, sr = librosa.load(abs_file_path, sr = 16000)
            dataset.append([data, file])
    
    print("Dataset 생성 완료")
    return pd.DataFrame(dataset,columns=['data', 'file_name'])

In [35]:
test_wav = test_dataset()

100%|██████████| 201/201 [00:10<00:00, 19.10it/s]

Dataset 생성 완료


In [38]:
test_wav.to_csv('test_wav.csv')

In [39]:
mfccs_2 = preprocess_train_dataset(test_wav.data)
mfccs_2 = np.array(mfccs_2)

In [40]:
# 모델 선언
model = RandomForestClassifier()

# 모델 학습
model.fit(mfccs, train_wav.label)

# 모델 예측
prediction = model.predict(mfccs_2)

test_wav['label'] = prediction

In [41]:
test_wav.head()

,data,file_name,label
0,"[0.00027645202, 0.00047594117, 0.0004510493, 0...",010.wav,3
1,"[0.00018057936, 0.00026723626, 0.00017172625, ...",024.wav,2
2,"[-9.729892e-05, -0.00013003226, -8.376751e-05,...",025.wav,0
3,"[-0.00012691804, -0.00019610435, -0.0001295761...",015.wav,3
4,"[0.00021518696, 0.00039842026, 0.00028533765, ...",078.wav,0


In [43]:
testset = test_wav[['file_name', 'label']]

pred_df = testset.copy()
pred_df = pred_df.sort_values(by=[pred_df.columns[0]], ascending=[True]).reset_index(drop=True)
pred_df.head()


,file_name,label
0,003.wav,0
1,008.wav,1
2,010.wav,3
3,015.wav,3
4,024.wav,2


In [44]:
submission = pd.read_csv('/content/gdrive/MyDrive/drive-download-20220614T024615Z-001/sample_submission.csv')
submission['label'] = pred_df['label']
submission.head()

,file_name,label
0,003.wav,0
1,008.wav,1
2,010.wav,3
3,015.wav,3
4,024.wav,2


In [45]:
submission.to_csv('submit.csv', index=False)
